# 0. Preparation

In [ ]:
#@title remove repos from disc
%cd /content
!rm -r airbnb_price
!rm -r yolov3

/content


In [ ]:
#@title Clone repo
!git clone https://github.com/dernameistegal/airbnb_price.git
!git clone https://github.com/jonaden94/yolov3.git


Cloning into 'airbnb_price'...
remote: Enumerating objects: 1082, done.
remote: Counting objects: 100% (1082/1082), done.
remote: Compressing objects: 100% (1027/1027), done.
remote: Total 1082 (delta 637), reused 293 (delta 49), pack-reused 0
Receiving objects: 100% (1082/1082), 8.46 MiB | 4.78 MiB/s, done.
Resolving deltas: 100% (637/637), done.
Cloning into 'yolov3'...
remote: Enumerating objects: 10005, done.
remote: Total 10005 (delta 0), reused 0 (delta 0), pack-reused 10005
Receiving objects: 100% (10005/10005), 9.49 MiB | 22.71 MiB/s, done.
Resolving deltas: 100% (6745/6745), done.


In [ ]:
#@title add paths to library search path
import sys 

sys.path.append("/content/airbnb_price/custom_functions")

In [ ]:
#@title Imports and drive
import os
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import requests
from tqdm import tqdm
import pandas as pd
from google.colab import files

# own modules
import general_utils as gu
import picture_model_utils as pu



from google.colab import drive

#@title Mount drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
#@title define device

# device
device = gu.get_device()
num_cpus = os.cpu_count()
print(num_cpus, 'CPUs available')

cuda available: True ; cudnn available: True ; num devices: 1
Using device Tesla P100-PCIE-16GB
4 CPUs available


In [ ]:
# remove sample pictures
!rm -r /content/yolov3/data/images

In [ ]:
# sim link to pictures in drive
!ln -s /content/drive/MyDrive/Colab/airbnb/munich/hostpics/hostpics_raw_jpeg/. /content/yolov3/data/images

In [ ]:
%cd yolov3

/content/yolov3


In [ ]:
%%capture
!pip install -U -r requirements.txt

In [ ]:
!python detect.py --weights yolov3.pt --source data/images --imgsz 224 --conf-thres 0.65

detect: weights=['yolov3.pt'], source=data/images, imgsz=[224, 224], conf_thres=0.65, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv3 🚀 v9.6.0-22-gef18067 torch 1.10.2+cu102 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

100% 119M/119M [00:07<00:00, 15.8MB/s]

Fusing layers... 
Model Summary: 261 layers, 61922845 parameters, 0 gradients, 156.1 GFLOPs
  0% 0/998 [00:00<?, ?it/s]image 1/998 /content/drive/.shortcut-targets-by-id/1KWmP0DEr72_amwQOSzblWXNNqlnaqU3z/munich/hostpics/hostpics_raw_jpeg/10213696.jpg: 224x224 Done. (0.013s)
  0% 1/998 [00:00<04:37,  3.60it/s]image 2/998 /content/drive/.shortcut-targets-by-id/1KWmP0DEr72_amwQOSzblWXNNqlnaqU3z/munich/hostpics/hostpics_raw_jpeg/10688793.jpg: 224x224 1 person, Don

# modify preds

In [ ]:
preds = pd.read_csv("/content/yolov3/runs/detect/exp/preds.txt", sep=":", header=None)

In [ ]:
preds.columns = ["id", "host_picture"]

preds.id = preds.id.str.replace("image.*?hostpics_raw_jpeg/", "", regex=True)
preds.id = preds.id.str.replace(".jpg", "", regex=True)

preds.host_picture = preds.host_picture.str.replace("224x224", "")
preds.host_picture = preds.host_picture.str[:-2]

preds.set_index("id", inplace=True)
preds["host_picture_binary"] = preds.host_picture.str.contains("person")
preds["host_picture_binary"] = preds["host_picture_binary"].astype("int")

In [ ]:
preds.head()

,host_picture,host_picture_binary
id,,
10213696,no object detected,0
10688793,1 person,1
11015816,1 person,1
11101904,1 person,1
11299203,"2 persons, 1 cup, 1 dining table",1


In [ ]:
preds.to_pickle("/content/drive/MyDrive/Colab/airbnb/munich/hostpics/hostpics_binary.pickle")